In [1]:
pip install opencv-python opencv-python-headless pytube


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python yt-dlp


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install streamlink


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Update this path to where you saved coco.names
coco_names_path = "/Users/chiragn/Desktop/TIMES SQUARE/coco.names"  # Change this to your path

with open(coco_names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]


In [5]:
pip install pafy youtube_dl opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade --force-reinstall "git+https://github.com/ytdl-org/youtube-dl.git"

  Cloning https://github.com/ytdl-org/youtube-dl.git to /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-req-build-o7hmt6_x
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-req-build-o7hmt6_x
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit c5098961b04ce83f4615f2a846c84f803b072639
  Preparing metadata (setup.py) ... done
  Created wheel for youtube-dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1969895 sha256=e1c3ca6a3a828de347263444bbfba849b93f3e7c1fb85cc0ddb814c7af9c55a0
  Stored in directory: /private/var/folders/47/bh11rgr97kv3n95hy38kv3540000gn/T/pip-ephem-wheel-cache-vxqr5uwq/wheels/91/cb/b5/629bb9aeb351f550a453ee0e4cccdea4c8cf4a16074cb0bd4e
Successfully built youtube-dl
  Attempting uninstall: youtube-dl
    Found existing installation: youtube-dl 2021.12.17
    Uninstalling youtube-dl-2021.12.17:
      Successfully uninstalled yo

In [7]:
pip install yt-dlp opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os

weights_path = "/Users/chiragn/Desktop/TIMES SQUARE/yolov3.weights"  # Update with your path
cfg_path = "/Users/chiragn/Desktop/TIMES SQUARE/yolov3.cfg"          # Update with your path

# Check if files exist
if not os.path.isfile(weights_path):
    print(f"Error: Weights file does not exist at {weights_path}")
else:
    print(f"Weights file exists at {weights_path}")

if not os.path.isfile(cfg_path):
    print(f"Error: Config file does not exist at {cfg_path}")
else:
    print(f"Config file exists at {cfg_path}")


Weights file exists at /Users/chiragn/Desktop/TIMES SQUARE/yolov3.weights
Config file exists at /Users/chiragn/Desktop/TIMES SQUARE/yolov3.cfg


In [9]:
pip install torch torchvision torchaudio ultralytics


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [11]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Make sure you have yolov8.pt in your current directory or specify the correct path


In [1]:
import cv2
import numpy as np
from datetime import datetime
import yt_dlp
import os

def load_yolo_model(weights_path, cfg_path):
    try:
        net = cv2.dnn.readNet(weights_path, cfg_path)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        return net, output_layers
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return None, None

def get_youtube_stream(url):
    ydl_opts = {'format': 'best'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info['url']

def log_output(message):
    with open("outputv8.txt", "a") as f:
        f.write(f"{message}\n")
    print(message)

def process_live_video(video_url):
    base_path = "/Users/chiragn/Desktop/TIMES SQUARE"
    weights_path = os.path.join(base_path, "yolov3.weights")
    cfg_path = os.path.join(base_path, "yolov3.cfg")
    coco_names_path = os.path.join(base_path, "coco.names")

    log_output(f"weights_path exists: {os.path.exists(weights_path)}, size: {os.path.getsize(weights_path) if os.path.exists(weights_path) else 'N/A'} bytes")
    log_output(f"cfg_path exists: {os.path.exists(cfg_path)}, size: {os.path.getsize(cfg_path) if os.path.exists(cfg_path) else 'N/A'} bytes")
    log_output(f"coco_names_path exists: {os.path.exists(coco_names_path)}, size: {os.path.getsize(coco_names_path) if os.path.exists(coco_names_path) else 'N/A'} bytes")

    net, output_layers = load_yolo_model(weights_path, cfg_path)

    if net is None:
        log_output("Failed to load YOLO model. Exiting.")
        return

    try:
        with open(coco_names_path, "r") as f:
            classes = [line.strip() for line in f.readlines()]
        log_output(f"Loaded {len(classes)} class names")
    except Exception as e:
        log_output(f"Error loading class names: {e}")
        return

    try:
        stream_url = get_youtube_stream(video_url)
        log_output(f"Successfully retrieved stream URL: {stream_url[:50]}...")
    except Exception as e:
        log_output(f"Error getting stream URL: {e}")
        return

    cap = cv2.VideoCapture(stream_url)

    if not cap.isOpened():
        log_output("Error opening video stream or file")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            log_output("Failed to capture frame. Exiting.")
            break

        frame_count += 1
        if frame_count % 1 == 0:  # Process every 10th frame for more frequent updates
            # Resize frame for faster processing
            height, width = frame.shape[:2]
            new_height = 832  # Increased resolution
            new_width = int(width * (new_height / height))
            resized_frame = cv2.resize(frame, (new_width, new_height))

            blob = cv2.dnn.blobFromImage(resized_frame, 1/255.0, (416, 416), swapRB=True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.1 and classes[class_id] == 'person':  # Lower confidence threshold
                        center_x = int(detection[0] * new_width)
                        center_y = int(detection[1] * new_height)
                        w = int(detection[2] * new_width)
                        h = int(detection[3] * new_height)

                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.3)  # Adjusted NMS parameters

            head_count = len(indexes)

            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    color = (0, 255, 0)
                    cv2.rectangle(resized_frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(resized_frame, f"{label} {confidences[i]:.2f}", (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Additional head detection using Haar Cascade
            gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
            haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(20, 20))
            
            for (x, y, w, h) in faces:
                cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                head_count += 1

            # Upper body detection
            upper_body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')
            upper_bodies = upper_body_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(30, 30))
            
            for (x, y, w, h) in upper_bodies:
                cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                head_count += 1

            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_output(f"[{current_time}] Head Count: {head_count}")

            cv2.putText(resized_frame, f"Head Count: {head_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Live Video", resized_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_video_url = "https://www.youtube.com/live/PQQ8EPRFLzo?si=vNT8GhTT89859XA8"
    process_live_video(live_video_url)

weights_path exists: True, size: 248007048 bytes
cfg_path exists: True, size: 8342 bytes
coco_names_path exists: True, size: 624 bytes
Loaded 80 class names
[youtube] Extracting URL: https://www.youtube.com/live/PQQ8EPRFLzo?si=vNT8GhTT89859XA8
[youtube] PQQ8EPRFLzo: Downloading webpage
[youtube] PQQ8EPRFLzo: Downloading ios player API JSON
[youtube] PQQ8EPRFLzo: Downloading web creator player API JSON
[youtube] PQQ8EPRFLzo: Downloading m3u8 information
[youtube] PQQ8EPRFLzo: Downloading m3u8 information
Successfully retrieved stream URL: https://manifest.googlevideo.com/api/manifest/hls_...
[2024-10-01 10:43:04] Head Count: 67
[2024-10-01 10:43:05] Head Count: 67
[2024-10-01 10:43:05] Head Count: 69
[2024-10-01 10:43:06] Head Count: 66


2024-10-01 10:43:06.343 python[20962:448280] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-01 10:43:06.343 python[20962:448280] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


[2024-10-01 10:43:06] Head Count: 67
[2024-10-01 10:43:07] Head Count: 64
[2024-10-01 10:43:07] Head Count: 65
[2024-10-01 10:43:08] Head Count: 65
[2024-10-01 10:43:08] Head Count: 62
[2024-10-01 10:43:09] Head Count: 63
[2024-10-01 10:43:10] Head Count: 64
[2024-10-01 10:43:10] Head Count: 58
[2024-10-01 10:43:11] Head Count: 57
[2024-10-01 10:43:11] Head Count: 59
[2024-10-01 10:43:12] Head Count: 61
[2024-10-01 10:43:12] Head Count: 61
[2024-10-01 10:43:13] Head Count: 69
[2024-10-01 10:43:13] Head Count: 67
[2024-10-01 10:43:14] Head Count: 68
[2024-10-01 10:43:14] Head Count: 63
[2024-10-01 10:43:15] Head Count: 59
[2024-10-01 10:43:15] Head Count: 59
[2024-10-01 10:43:16] Head Count: 71
[2024-10-01 10:43:16] Head Count: 69
[2024-10-01 10:43:17] Head Count: 68
[2024-10-01 10:43:17] Head Count: 64
[2024-10-01 10:43:18] Head Count: 65
[2024-10-01 10:43:18] Head Count: 64
[2024-10-01 10:43:18] Head Count: 53
[2024-10-01 10:43:19] Head Count: 57
[2024-10-01 10:43:19] Head Count: 56
[

KeyboardInterrupt: 

: 